In [1]:
import pandas as pd
import json
import tqdm
from google.colab import drive

In [2]:
drive.mount("/drive")

Mounted at /drive


In [20]:
data_fname = "/drive/MyDrive/bert_herb_drug/Relation_extraction_herb_chemical.jsonl"

In [21]:
all_data = []

In [22]:
f = open(data_fname, "r")
line = f.readline()

In [23]:
j_data = json.loads(line)

In [24]:
relations = "relations"
head_span = "head_span"
child_span = "child_span"
label = "label"
start = "start"
end = "end"
text = "text"

In [25]:
j_data["relations"][0]

{'child': 87,
 'child_span': {'end': 392,
  'label': 'HERB',
  'start': 375,
  'token_end': 87,
  'token_start': 86},
 'color': '#c5bdf4',
 'head': 82,
 'head_span': {'end': 349,
  'label': 'CHEMICAL',
  'start': 337,
  'token_end': 82,
  'token_start': 82},
 'label': 'POSITIVE'}

# Data Formatting

In [26]:
data = (j_data[text], j_data[text][j_data[relations][0][head_span][start]:j_data[relations][0][head_span][end]], j_data[relations][0][label])

In [17]:
data

('), was given to the rabbits and the blood samples were withdrawn at different time intervals (0, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 12, 24 and 36 h) from marginal ear vein after dosing and theophylline in plasma was analyzed by hplc method. results: it was observed that there a significant differences in the cmax, auc, aumc, t1/2, and mrt of theophylline when coadministered with commiphora myrrha which indicate that the herb affect the metabolism and elimination when coadministered with theophylline . conclusion: our results suggested that concurrent use of investigated herb alters the pharmacokinetics of theophylline.',
 'theophylline',
 'POSITIVE')

In [27]:
f.seek(0)
all_data_dict = {i: j_data for i, j_data in enumerate(f)}
all_data_dict[0]

'{"text":"), was given to the rabbits and the blood samples were withdrawn at different time intervals (0, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 12, 24 and 36 h) from marginal ear vein after dosing and theophylline in plasma was analyzed by hplc method. results: it was observed that there a significant differences in the cmax, auc, aumc, t1/2, and mrt of theophylline when coadministered with commiphora myrrha which indicate that the herb affect the metabolism and elimination when coadministered with theophylline . conclusion: our results suggested that concurrent use of investigated herb alters the pharmacokinetics of theophylline.","_input_hash":447283788,"_task_hash":-565715972,"spans":[{"start":182,"end":194,"token_start":50,"token_end":50,"label":"CHEMICAL"},{"start":337,"end":349,"token_start":82,"token_end":82,"label":"CHEMICAL"},{"start":375,"end":392,"token_start":86,"token_end":87,"label":"HERB"},{"start":485,"end":497,"token_start":101,"token_end":101,"label":"CHEMICAL"},{"start":570,"

# Preparing all Data

In [28]:
def get_mention(text: str, span: dict) -> str:
  return text[span[start]: span[end]]

In [36]:
from tqdm import tqdm
f.seek(0)
all_data = []
for line in tqdm(f):
  j_data = json.loads(line)
  curr_text = j_data[text]
  for relation in j_data[relations]:
    entity_mention_1 = get_mention(curr_text, relation[head_span])
    entity_mention_2 = get_mention(curr_text, relation[child_span])
    all_data.append((curr_text, entity_mention_1, relation[head_span][start], relation[head_span][end], relation[head_span][label], entity_mention_2, relation[child_span][start], relation[child_span][end], relation[child_span][label], relation[label]))

165it [00:00, 9190.71it/s]


In [37]:
total_data = len(all_data)
total_data

433

Removing unwanted data

In [38]:
# Index 9 is reletion type
all_data = [tup for tup in all_data if tup[9] != "COREF"]
total_data = len(all_data)
total_data

424

In [39]:
# Index 4 and 8 is entity labels for head and child
all_data = [tup for tup in all_data if tup[4] != tup[8]]
total_data = len(all_data)
total_data

415

In [40]:
# Index 4 and 8 is entity labels for head and child
all_data = [tup for tup in all_data if not ((tup[4] == "HERB_COMPOUND" and tup[8] == "HERB") or (tup[8] == "HERB_COMPOUND" and tup[4] == "HERB"))]
total_data = len(all_data)
total_data

415

In [41]:
valid_size = int(len(all_data) * .1)
valid_size

41

In [42]:
train_data = all_data[:total_data - valid_size]
len(train_data)

374

In [43]:
valid_data = all_data[total_data - valid_size:]
len(valid_data)

41

In [48]:
train = train_data[0]

curr_text = train[0]
entity_mention_1 = train[1]
entity_mention_1_start = train[2]
entity_mention_1_end = train[3]
#entity_mention_1_label = train[4]
entity_mention_2 = train[5]
entity_mention_2_start = train[6]
entity_mention_2_end = train[7]
# entity_mention_2_label = train[8]
curr_label = train[9]
curr_text = curr_text[:entity_mention_1_start:] + f"<e1> {entity_mention_1} </e1>" + curr_text[entity_mention_1_end:]
curr_text = curr_text[:entity_mention_2_start:] + f" <e2> {entity_mention_2} </e2> " + curr_text[entity_mention_2_end:]

first_entity_start = min(entity_mention_1_start, entity_mention_2_start)
last_entity_end = max(entity_mention_1_end, entity_mention_2_end)

sentence_start = curr_text.rfind(".", 0, first_entity_start) + 1
sentence_end = curr_text.find(".", last_entity_end) + 1

curr_text[sentence_start:sentence_end]

' results: it was observed that there a significant differences in the cmax, auc, aumc, t1/2, and mrt of <e1> theophylline </e1> when coadminis <e2> commiphora myrrha </e2> hora myrrha which indicate that the herb affect the metabolism and elimination when coadministered with theophylline .'

In [49]:
new_train = []
for train in tqdm(train_data):
  curr_text = train[0]
  entity_mention_1 = train[1]
  entity_mention_1_start = train[2]
  entity_mention_1_end = train[3]
  #entity_mention_1_label = train[4]
  entity_mention_2 = train[5]
  entity_mention_2_start = train[6]
  entity_mention_2_end = train[7]
  #entity_mention_2_label = train[8]
  curr_label = train[9]
  curr_text = curr_text[:entity_mention_1_start:] + f" <e1> {entity_mention_1} </e1> " + curr_text[entity_mention_1_end:]
  curr_text = curr_text[:entity_mention_2_start:] + f" <e2> {entity_mention_2} </e2> " + curr_text[entity_mention_2_end:]

  first_entity_start = min(entity_mention_1_start, entity_mention_2_start)
  last_entity_end = max(entity_mention_1_end, entity_mention_2_end)

  sentence_start = curr_text.rfind(".", 0, first_entity_start) + 1
  sentence_end = curr_text.find(".", last_entity_end) + 1

  new_train.append((curr_text[sentence_start:sentence_end], entity_mention_1, entity_mention_2, curr_label))

100%|██████████| 374/374 [00:00<00:00, 81080.77it/s]


In [50]:
new_train[10]

('aim of the study: the antithrombotic effect of danggui and honghua, herbs commonly used in traditional chinese medicine to treat the syndrome corresponding to vascular thrombosis, and  <e1> their </e1>  potential inter <e2> clopidogrel </e2> h clopidogrel were investigated.',
 'their',
 'clopidogrel',
 'SPECULATIVE')

In [52]:
train_df = pd.DataFrame(new_train, columns=["sentences", "entity_1_mention", "entity_2_mention",  "type"])
train_df

,sentences,entity_1_mention,entity_2_mention,type
0,results: it was observed that there a signifi...,theophylline,commiphora myrrha,POSITIVE
1,conclusion: our results suggested that concur...,herb,theophylline,POSITIVE
2,results: it was observed that there a signifi...,commiphora myrrha,theophylline,POSITIVE
3,results: as compared with <e1> sesame </ <e2...,sesame,glibenclamide,POSITIVE
4,"01 vs <e1> sesame </e1> oil monotherapy, <e2...",sesame,glibenclamide,POSITIVE
...,...,...,...,...
369,conclusion: the radix bupleuri and <e1> vine...,vinegar-baked radix bupleuri,cyp2c19,POSITIVE
370,conclusion: the <e1> radix bupleuri </e1> a...,radix bupleuri,cyp2c19,POSITIVE
371,conclusion: the <e1> radix bupleuri </e1> a...,radix bupleuri,cyp2c9,POSITIVE
372,the pharmacokinetics of <e1> caffeine < <e2> ...,caffeine,radix bupleuri,POSITIVE


In [55]:
valid_df = pd.DataFrame(valid_data, columns=["sentences", "entity_1_mention", "start1", "end1", "entity_1_label", "entity_2_mention", "start2", "end2", "entity_2_label", "type"])
valid_df.drop(["start1", "start2", "end1", "end2"], axis=1, inplace=True)
valid_df

,sentences,entity_1_mention,entity_1_label,entity_2_mention,entity_2_label,type
0,the pharmacokinetics of tolbutamide in all vin...,tolbutamide,CHEMICAL,vinegar-baked radix bupleuri,HERB,NEGATIVE
1,the pharmacokinetics of tolbutamide in all vin...,radix bupleuri,HERB,tolbutamide,CHEMICAL,POSITIVE
2,the plasma concentrations of the six probes we...,midazolam,CHEMICAL,radix bupleuri,HERB,POSITIVE
3,the plasma concentrations of the six probes we...,midazolam,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
4,the plasma concentrations of the six probes we...,dextromethorphan,CHEMICAL,radix bupleuri,HERB,POSITIVE
5,the plasma concentrations of the six probes we...,dextromethorphan,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
6,the plasma concentrations of the six probes we...,chlorzoxazone,CHEMICAL,radix bupleuri,HERB,POSITIVE
7,the plasma concentrations of the six probes we...,chlorzoxazone,CHEMICAL,vinegar-baked radix bupleuri,HERB,POSITIVE
8,the reason of different therapeutic effects of...,radix bupleuri,HERB,cyp2c9,CHEMICAL,POSITIVE
9,the reason of different therapeutic effects of...,vinegar-baked radix bupleuri,HERB,cyp2c19,CHEMICAL,POSITIVE


In [56]:
from sklearn import model_selection
def randomKFold_func(df, n_splits=5):
    # creat a new colunm called kfold and fill it with -1
    df['kfold'] = -1
    # randomize the rows of the data
    df = df.sample(frac=1).reset_index(drop=True)
    # initiate the kfold class for model selection module
    kf = model_selection.KFold(n_splits=n_splits)
    # fill the new kfold column
    for f, (t_, v_) in enumerate(kf.split(X=df)):
        df.loc[v_, 'kfold'] = f 
    return df

dataset_df_kfold = randomKFold_func(train_df, n_splits=5)

In [57]:
dataset_df_kfold

,sentences,entity_1_mention,entity_2_mention,type,kfold
0,interactions of several commonly used herbal m...,milk thistle,indinavir,POSITIVE,0
1,"0 years, and their most-used medications were ...",mint,atorvastatin,SPECULATIVE,0
2,<e1> khat </e1> chewing has been shown to re...,khat,amoxicillin,POSITIVE,0
3,conclusion: the growth inhibitory activity of...,t. bellerica,doxorubicin,POSITIVE,0
4,by taking advantage of the high protein bindin...,chan su,digoxin,POSITIVE,0
...,...,...,...,...,...
369,aim of the study: the antithrombotic effect of...,honghua,clopidogrel,SPECULATIVE,4
370,63 ng/ml digoxin) increase in digoxin concentr...,asian ginseng,digoxin,POSITIVE,4
371,as hops ( <e1> humulus lupulus l. </e1> ) are ...,humulus lupulus l.,drug,POSITIVE,4
372,conclusions: the present investigation demons...,rhodiola rosea l.,mao a,POSITIVE,4


In [58]:
path_save ="/drive/MyDrive/bert_herb_drug/datasets/"

In [59]:
dataset_df_kfold.to_csv(path_save + "train_new.csv")
valid_df.to_csv(path_save + "valid_new.csv")